In [1]:
import numpy as np
import csv
import GPy
from matplotlib import pyplot as plt
import pandas as pd
import warnings
warnings.simplefilter("error", RuntimeWarning)

In [2]:
def cnt_rmse(y, y_hat):
    return np.sqrt(((y_hat.flatten()-y.flatten())**2).mean())

In [7]:
fname = '/home/chun/Desktop/gpr_testing/concrete_slump/slump_test.data'


df = pd.read_csv(fname, header=0)
tmp = df.drop(df.columns[[0,9,10]], axis=1)

tmp.head(5)

,Cement,Slag,Fly ash,Water,SP,Coarse Aggr.,Fine Aggr.,SLUMP(cm)
0,273.0,82.0,105.0,210.0,9.0,904.0,680.0,23.0
1,163.0,149.0,191.0,180.0,12.0,843.0,746.0,0.0
2,162.0,148.0,191.0,179.0,16.0,840.0,743.0,1.0
3,162.0,148.0,190.0,179.0,19.0,838.0,741.0,3.0
4,154.0,112.0,144.0,220.0,10.0,923.0,658.0,20.0


In [4]:
np_df = tmp.values

In [5]:
X = np_df[:,:-1]
Y = np.reshape(np_df[:,-1], (len(np_df), 1))
X.shape

(103, 7)

In [82]:
kernel = GPy.kern.RBF(input_dim = 7, variance = 1., lengthscale = 1., ARD = True)
# kernel = GPy.kern.RBF(input_dim = 7, variance = 1., lengthscale = 1.)
optimize = True
# optimize = False

data_idx = np.arange(len(X))
np.random.shuffle(data_idx)

test_size = int(np.floor(X.shape[0]/10))
split_idx = [test_size*i for i in range(1,10)]
test_idx_list = np.split(data_idx, split_idx)

train_size = 0
train_size = 50

err_list = []
for test_idx in test_idx_list:
    all_idx = np.arange(len(X))
    tmp_train_idx = np.delete(all_idx, test_idx)
    test_X = X[test_idx]
    if train_size == 0:
        new_train_idx = tmp_train_idx
    else:
        np.random.shuffle(tmp_train_idx)
        new_train_idx = tmp_train_idx[:train_size]

    new_X = X[new_train_idx]
#     print(test_X.shape)
#     print(new_X.shape)
    
    # normalize y 
    Y_mean = Y - Y[new_train_idx].mean()
    test_y = Y_mean[test_idx]
    new_y = Y_mean[new_train_idx]
#     print(Y)
#     Y = Y - Y[new_train_idx].mean()
#     print(Y)
#     test_y = Y[test_idx]
#     new_y = Y[new_train_idx]
    
    try:
#         m = GPy.models.GPRegression(new_X, new_y, kernel, normalizer=True)
        m = GPy.models.GPRegression(new_X, new_y, kernel, normalizer=False)
    except RuntimeWarning as e:
        print(e, '0')
#         continue
    if optimize:
        try:
            m.optimize(max_iters=50, optimizer="lbfgs")
    #         m.optimize(max_iters=100)
        except RuntimeWarning as e:
            print(e, '1')
#             continue
    try:
        y_hat = m.predict(test_X)[0]
    except RuntimeWarning as e:
        print(e, '2')
#         continue
    try:
        err = cnt_rmse(y_hat, test_y)
    except RuntimeWarning as e:
        print(e, '3')
#         continue
    
    err_list.append(err)

print(sum(err_list)/len(err_list))

8.537104943048647


In [ ]:
print(err_list)

In [ ]:
p = np.arange(101)
test_size = int(np.floor(len(p)/10))
split_idx = [test_size*i for i in range(1,10)]
print(split_idx)
i_test_idx = np.split(p, split_idx)
print(len(i_test_idx), '\n')
for idx in i_test_idx:
    print(len(idx))
    
a = np.array([1,2,3,4,5])
print(np.delete(a, [1,2,3]))
a